In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Using sqlalchemy to query EzPresto

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import event
 
%update_token
 
auth_token=os.environ["AUTH_TOKEN"]
session = requests.Session()
session.headers["Authorization"] = "Bearer " + f"{auth_token}"
 
engine = create_engine(
        "presto://ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local:8081",
        connect_args={"protocol": "https","requests_kwargs": {"verify": False}, "requests_session": session},
    )

In [ ]:
QUERY = "SELECT * from awspsql.public.user limit 10"
 
with engine.connect() as connection:
    with connection.begin():
        # run statements in a "begin once" block
        result = connection.execute(text(QUERY))
        for row in result:
            print(row)

In [ ]:
QUERY = "SELECT * from awspsql.public.review limit 10"
 
with engine.connect() as connection:
    with connection.begin():
        # run statements in a "begin once" block
        result = connection.execute(text(QUERY))
        for row in result:
            print(row)

## Write a dummy record into `review` table

In [ ]:
with engine.connect() as connection:
    with connection.begin():
        connection.execute(
            text("""INSERT INTO awspsql.public.review (
                review_id,
                user_id,
                business_id,
                stars,
                useful,
                funny,
                cool,
                text,
                date)
            VALUES (
                :review_id,
                :user_id,
                :business_id,
                :stars,
                :useful,
                :funny,
                :cool,
                :text,
                :date
                )"""),
            [{"review_id":"111",
                "user_id":"111",
                "business_id":"111",
                "stars":5,
                "useful":5,
                "funny":5,
                "cool":5,
                "text":"Awesome demo",
                "date": None}],
        )

In [ ]:
QUERY = "SELECT * from awspsql.public.review where review_id='111'"
 
with engine.connect() as connection:
    with connection.begin():
        result = connection.execute(text(QUERY))
        for row in result:
            print(row)

# Using Spark to Query EzPresto
Switch to a pyspark kernel for running this section

In [ ]:
%manage_spark

In [ ]:
dfReader = spark.read.format("EzPresto")
dfReader.option("presto_url", "https://ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local:8081")
dfReader.option("dal_url", "https://ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local:9090")
dfReader.option("ignore_ssl_check", "true")
dfReader.option("query", "select * from awspsql.public.user limit 10")

df = dfReader.load()
df.show()

In [ ]:
dfReader = spark.read.format("EzPresto")
dfReader.option("presto_url", "https://ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local:8081")
dfReader.option("dal_url", "https://ezpresto-sts-mst-0.ezpresto-svc-hdl.ezpresto.svc.cluster.local:9090")
dfReader.option("ignore_ssl_check", "true")
dfReader.option("query", "select * from awspsql.public.review limit 10")

df = dfReader.load()
df.show()